In [1]:
 {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "\n",
    "```python\n",
    "import requests\n",
    "from bs4 import BeautifulSoup\n",
    "\n",
    "def get_most_recent_articles(url):\n",
    "    response = requests.get(url)\n",
    "    soup = BeautifulSoup(response.text, 'html.parser')\n",
    "    articles = soup.find_all('article', class_='news-item') # Adjust the selector based on the website's structure\n",
    "    return articles[:10]\n",
    "\n",
    "url = 'https://www.mediapool.bg/news/'\n",
    "articles = get_most_recent_articles(url)\n",
    "```\n"
   ]
  }

NameError: name 'null' is not defined